In [2]:
!pip install py2neo
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177 kB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 173 kB 5.0 MB/s 
  Created wheel for neo4j: filename=neo4j-5.1.0-py3-none-any.whl size=248097 sha256=01bb84692413895a62ff50961b5a51f207cf956922eb89a447f04e2e846bc545
  Stored in directory: /root/.cache/pip/wheels/64/96/da/c30ec8577a970eadecc436103fbcbf7af1d6398d0ebe32cbab
Successfully built neo4j


In [3]:
import requests
import json
import csv
import io
from py2neo import Graph
from neo4j import GraphDatabase, basic_auth

In [4]:
def SubmitPMIDList(Pmid,Format,Bioconcept):

    # json = {"Pmids": Pmid}

    json = {}

    #
    # load pmids
    #
    with io.open(Inputfile,'r',encoding="utf-8") as file_input:
        json = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
    if Bioconcept != "": 
        json["concepts"] = Bioconcept.split(",")
    
    print(json)
    r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/" + Format , json = json)
    
    res = r.json()
    print(res)

    pmid = res['id']
    res0 = res['passages'][0]['annotations']
    res1 = res['passages'][1]['annotations']
    
    result = {}
    result = res0 + res1
    
    for i in result:
        i['Pmid'] = Pmid
    return result
    
    if r.status_code != 200 :
        print ("[Error]: HTTP code "+ str(r.status_code))
    else:
        return result

In [12]:
Inputfile = "./pmid"
Format = "biocjson"
Bioconcept = ""
res_json=[]

In [13]:
with io.open(Inputfile,'r',encoding="utf-8") as file_input:
    pmidlist = {"pmids": [pmid.strip() for pmid in file_input.readlines()]}
    
for i in pmidlist['pmids']:
    res_json.extend(SubmitPMIDList([i], Format, Bioconcept))

{'pmids': ['35900868']}
{'_id': '35900868|None', 'id': '35900868', 'infons': {}, 'passages': [{'infons': {'journal': 'J Clin Invest;2022Jul28. doi:10.1172/JCI161908', 'year': '2022', 'type': 'title', 'authors': 'Lone MA, Aaltonen MJ, Zidell A, Pedro HF, Morales Saute JA, Mathew S, Mohassel P, Bonnemann CG, Shoubridge EA, Hornemann T, ', 'section': 'Title'}, 'offset': 0, 'text': 'SPTLC1 variants associated with ALS produce distinct sphingolipid signatures through impaired interaction with ORMDL proteins.', 'sentences': [], 'annotations': [{'id': '2', 'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'}, 'text': 'SPTLC1', 'locations': [{'offset': 0, 'length': 6}]}, {'id': '3', 'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'}, 'text': 'sphingolipid', 'locations': [{'offset': 53, 'length': 12}]}], 'relations': []}, {'infons': {'type': 'abstract', 'section': 'Abstract'}, 'offset': 127, 'text': 'Amyotrophic lateral sclerosis (ALS) is a progressive neurodeg

In [ ]:
x = {
  "name": "John",
  "age": 30,
  "married": True,
  "divorced": False,
  "children": ("Ann","Billy"),
  "pets": None,
  "cars": [
    {"model": "BMW 230", "mpg": 27.5},
    {"model": "Ford Edge", "mpg": 24.1}
  ]
}

# print list of "cars" "model"s
print([x['cars'][i]['model'] for i in range(len(x['cars']))])

In [16]:
res_json

[{'id': '2',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 0, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '3',
  'infons': {'identifier': 'MESH:D013107', 'type': 'Chemical'},
  'text': 'sphingolipid',
  'locations': [{'offset': 53, 'length': 12}],
  'Pmid': ['35900868']},
 {'id': '31',
  'infons': {'identifier': 'MESH:D000690', 'type': 'Disease'},
  'text': 'Amyotrophic lateral sclerosis',
  'locations': [{'offset': 127, 'length': 29}],
  'Pmid': ['35900868']},
 {'id': '32',
  'infons': {'identifier': 'MESH:D019636', 'type': 'Disease'},
  'text': 'neurodegenerative disease',
  'locations': [{'offset': 180, 'length': 25}],
  'Pmid': ['35900868']},
 {'id': '33',
  'infons': {'identifier': '10558', 'type': 'Gene', 'ncbi_homologene': '4681'},
  'text': 'SPTLC1',
  'locations': [{'offset': 248, 'length': 6}],
  'Pmid': ['35900868']},
 {'id': '34',
  'infons': {'identifier': '189', 'type': 'Gene', 'ncbi_homo

In [28]:
class my_dictionary(dict): 
    
    def __init__(self): 
        self = dict() 
          
    def add(self, key, value): 
        self[key] = value 

dict_list = []

for data in res_json:
    dict_obj = my_dictionary() 
    
    for key, val in data.items():
        if type(val) == dict:    
            for k, v in val.items():
                dict_obj.add(k, v)
        elif type(val) == list:
            for k, v in val[0].items():
                dict_obj.add(k, v)
        else:
            dict_obj.add(key, val)


    dict_list.append(dict_obj)

result = []
    
for item in dict_list:
    item = {key: item[key] for key in item.keys() & {'pmid', 'type', 'text'}}
    result.append(item)
    

AttributeError: ignored

In [ ]:
result = [dict(t) for t in {tuple(d.items()) for d in result}]

In [ ]:
# data_file = open('./response.csv', 'w', newline='')
# csv_writer = csv.writer(data_file)

# count = 0
 
# for i in result:
#     if count == 0:
#         header = i.keys()
#         csv_writer.writerow(header)
#         count += 1
 
#     csv_writer.writerow(i.values())
 
# data_file.close()

In [ ]:
import requests
import re
import hashlib
from spacy import Language
from typing import List
import spacy

from spacy.tokens import Doc, Span

from transformers import pipeline


def call_wiki_api(item):
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    print(data)
    return data['search'][0]['id']

call_wiki_api("usa")

{'searchinfo': {'search': 'usa'}, 'search': [{'id': 'Q30', 'title': 'Q30', 'pageid': 126, 'display': {'label': {'value': 'United States of America', 'language': 'en'}, 'description': {'value': 'country in North America', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q30', 'concepturi': 'http://www.wikidata.org/entity/Q30', 'label': 'United States of America', 'description': 'country in North America', 'match': {'type': 'alias', 'language': 'en', 'text': 'USA'}, 'aliases': ['USA']}, {'id': 'Q9212', 'title': 'Q9212', 'pageid': 10572, 'display': {'label': {'value': 'United States Army', 'language': 'en'}, 'description': {'value': 'land service branch of the United States Armed Forces', 'language': 'en'}}, 'repository': 'wikidata', 'url': '//www.wikidata.org/wiki/Q9212', 'concepturi': 'http://www.wikidata.org/entity/Q9212', 'label': 'United States Army', 'description': 'land service branch of the United States Armed Forces', 'match': {'type': 'alias', 'langu

'Q30'

In [ ]:
def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})

    return triplets


In [ ]:
@Language.factory(
    "rebel",
    requires=["doc.sents"],
    assigns=["doc._.rel"],
    default_config={
        "model_name": "Babelscape/rebel-large",
        "device": 0,
    },
)
class RebelComponent:
    def __init__(
        self,
        nlp,
        name,
        model_name: str,
        device: int,
    ):
        assert model_name is not None, ""
        self.triplet_extractor = pipeline("text2text-generation", model=model_name, tokenizer=model_name, device=device)
        self.entity_mapping = {}
        # Register custom extension on the Doc
        if not Doc.has_extension("rel"):
          Doc.set_extension("rel", default={})

    def get_wiki_id(self, item: str):
        mapping = self.entity_mapping.get(item)
        if mapping:
          return mapping
        else:
          res = call_wiki_api(item)
          self.entity_mapping[item] = res
          return res

    
    def _generate_triplets(self, sent: Span) -> List[dict]:
          output_ids = self.triplet_extractor(sent.text, return_tensors=True, return_text=False)[0]["generated_token_ids"]["output_ids"]
          extracted_text = self.triplet_extractor.tokenizer.batch_decode(output_ids[0])
          extracted_triplets = extract_triplets(extracted_text[0])
          return extracted_triplets

    def set_annotations(self, doc: Doc, triplets: List[dict]):
        for triplet in triplets:

            # Remove self-loops (relationships that start and end at the entity)
            if triplet['head'] == triplet['tail']:
                continue

            # Use regex to search for entities
            head_span = re.search(triplet["head"], doc.text)
            tail_span = re.search(triplet["tail"], doc.text)

            # Skip the relation if both head and tail entities are not present in the text
            # Sometimes the Rebel model hallucinates some entities
            if not head_span or not tail_span:
              continue

            index = hashlib.sha1("".join([triplet['head'], triplet['tail'], triplet['type']]).encode('utf-8')).hexdigest()
            if index not in doc._.rel:
                # Get wiki ids and store results
                doc._.rel[index] = {"relation": triplet["type"], "head_span": {'text': triplet['head'], 'id': self.get_wiki_id(triplet['head'])}, "tail_span": {'text': triplet['tail'], 'id': self.get_wiki_id(triplet['tail'])}}

    def __call__(self, doc: Doc) -> Doc:
        for sent in doc.sents:
            sentence_triplets = self._generate_triplets(sent)
            self.set_annotations(doc, sentence_triplets)
        return doc

In [ ]:
DEVICE = -1 # Number of the GPU, -1 if want to use CPU

# Add coreference resolution model
coref = spacy.load('en_core_web_sm', disable=['ner', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer'])
coref.add_pipe(
    "xx_coref", config={"chunk_size": 2500, "chunk_overlap": 2, "device": DEVICE})

# Define rel extraction model

rel_ext = spacy.load('en_core_web_sm', disable=['ner', 'lemmatizer', 'attribute_rules', 'tagger'])
rel_ext.add_pipe("rebel", config={
    'device':DEVICE, # Number of the GPU, -1 if want to use CPU
    'model_name':'Babelscape/rebel-large'} # Model used, will default to 'Babelscape/rebel-large' if not given
    )



/Users/ryan/Git/KG_ALS_Exercise/venv/lib/python3.9/site-packages/spacy/util.py:865: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


ValueError: [E002] Can't find factory for 'xx_coref' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, lemmatizer, trainable_lemmatizer, entity_linker, ner, beam_ner, entity_ruler, tagger, morphologizer, senter, sentencizer, textcat, spancat, future_entity_ruler, span_ruler, textcat_multilabel, rebel, en.lemmatizer

In [ ]:
input_text = "Christian Drosten works in Germany. He likes to work for Google."

coref_text = coref(input_text)._.resolved_text

doc = rel_ext(coref_text)

for value, rel_dict in doc._.rel.items():
    print(f"{value}: {rel_dict}")